In [33]:
import pandas as pd
from google.colab import drive
from itertools import combinations

drive.mount('/content/drive')

path = '/content/drive/My Drive/NBA_Stats.xlsx'

# Read the Excel file into a DataFrame.
df = pd.read_excel(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
current_season = []

for i, player in df.iterrows():
  if(player["SeasonStartYear"] == 2018 and player["SeasonType"] == "Regular Season"):
      current_season.append(player)

df_2223 = pd.DataFrame(current_season)

In [35]:
df_2223

,Rank,Year,SeasonStartYear,SeasonType,PlayerID,Player,TeamID,Team,GamesPlayed,MinutesPlayed,...,Rebounds,Assists,Steals,Blocks,Turnovers,PersonalFouls,PointsScored,Efficency,ASTperTOV,STLperTOV
29485,1,2018-19,2018,Regular Season,201935,James Harden,1610612745,HOU,78,2867,...,518,586,158.0,58.0,387.0,244,2818,2581,1.51,0.41
29486,2,2018-19,2018,Regular Season,202331,Paul George,1610612760,OKC,77,2841,...,628,318,170.0,34.0,205.0,214,2159,2110,1.55,0.83
29487,3,2018-19,2018,Regular Season,202689,Kemba Walker,1610612766,CHA,82,2863,...,361,484,102.0,34.0,211.0,131,2102,1849,2.29,0.48
29488,4,2018-19,2018,Regular Season,203078,Bradley Beal,1610612764,WAS,82,3028,...,411,448,121.0,58.0,224.0,226,2099,1982,2.00,0.54
29489,5,2018-19,2018,Regular Season,203081,Damian Lillard,1610612757,POR,80,2838,...,371,551,88.0,34.0,212.0,148,2067,2002,2.60,0.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30010,521,2018-19,2018,Regular Season,1628424,Kobi Simmons,1610612739,CLE,1,2,...,0,0,0.0,0.0,0.0,0,0,0,0.00,0.00
30011,521,2018-19,2018,Regular Season,1627855,Okaro White,1610612764,WAS,3,6,...,2,0,0.0,0.0,0.0,0,0,0,0.00,0.00
30012,521,2018-19,2018,Regular Season,1629093,Tyler Davis,1610612760,OKC,1,1,...,1,0,0.0,0.0,0.0,0,0,0,0.00,0.00
30013,521,2018-19,2018,Regular Season,1627755,Tyler Ulis,1610612741,CHI,1,1,...,0,0,0.0,0.0,0.0,0,0,0,0.00,0.00


<br>

### Find pairs of players whose average PTS/gm stats are similar based on a defined margin.

Generate list of combinations of players, without duplicates, to create subset of pairs whose average assists per game are within a defined margin of each other.

In [43]:
player_pairs = list(combinations(df_2223.itertuples(index=False), 2))
df_pts_pairs = pd.DataFrame()

margin = 0.75

for pair in player_pairs:
  playerAstatpg = pair[0].PointsScored / pair[0].GamesPlayed
  playerBstatpg = pair[1].PointsScored / pair[1].GamesPlayed
  if(pair[0].GamesPlayed > 10 and pair[1].GamesPlayed > 10):
    answer = pair[0].Player if playerAstatpg > playerBstatpg else pair[1].Player
    diff = abs(playerBstatpg - playerAstatpg)
    if diff < margin and diff >= 0.25 and playerAstatpg > 7:
      df_pts_pairs = pd.concat([df_pts_pairs, pd.DataFrame.from_records([{
      "PlayerAId": pair[0].PlayerID,
      "PlayerAName": pair[0].Player,
      "PlayerAStatperGame": playerAstatpg,
      "PlayerATeam": pair[0].Team,
      "PlayerBId": pair[1].PlayerID,
      "PlayerBName": pair[1].Player,
      "PlayerBStatperGame": playerBstatpg,
      "PlayerBTeam": pair[1].Team,
      "SeasonStart": pair[1].SeasonStartYear,
      "Answer": answer,
      "StatCategory": "PTS",
      "Interval": diff

      }])])

print(len(df_pts_pairs))

2588


In [45]:
df_pts_pairs

,PlayerAId,PlayerAName,PlayerAStatperGame,PlayerATeam,PlayerBId,PlayerBName,PlayerBStatperGame,PlayerBTeam,SeasonStart,Answer,StatCategory,Interval
0,202331,Paul George,28.038961,OKC,203507,Giannis Antetokounmpo,27.694444,MIL,2018,Paul George,PTS,0.344517
0,202331,Paul George,28.038961,OKC,203954,Joel Embiid,27.515625,PHI,2018,Paul George,PTS,0.523336
0,202331,Paul George,28.038961,OKC,2544,LeBron James,27.363636,LAL,2018,Paul George,PTS,0.675325
0,202689,Kemba Walker,25.634146,CHA,201142,Kevin Durant,25.987179,GSW,2018,Kevin Durant,PTS,0.353033
0,202689,Kemba Walker,25.634146,CHA,203076,Anthony Davis,25.928571,NOP,2018,Anthony Davis,PTS,0.294425
...,...,...,...,...,...,...,...,...,...,...,...,...
0,1626174,Christian Wood,8.190476,NOP,1628415,Dillon Brooks,7.500000,MEM,2018,Christian Wood,PTS,0.690476
0,2736,Luol Deng,7.136364,MIN,1628415,Dillon Brooks,7.500000,MEM,2018,Dillon Brooks,PTS,0.363636
0,2736,Luol Deng,7.136364,MIN,2747,JR Smith,6.727273,CLE,2018,Luol Deng,PTS,0.409091
0,1628365,Markelle Fultz,8.157895,ORL,1628415,Dillon Brooks,7.500000,MEM,2018,Markelle Fultz,PTS,0.657895


### Find pairs of players whose average AST/gm stats are similar based on a defined margin.

In [44]:
player_pairs = list(combinations(df_2223.itertuples(index=False), 2))
df_ast_pairs = pd.DataFrame()

margin = 2.5

for pair in player_pairs:
  playerAstatpg = pair[0].Assists / pair[0].GamesPlayed
  playerBstatpg = pair[1].Assists / pair[1].GamesPlayed
  if(pair[0].GamesPlayed > 10 and pair[1].GamesPlayed > 10):
    answer = pair[0].Player if playerAstatpg > playerBstatpg else pair[1].Player
    diff = abs(playerBstatpg - playerAstatpg)
    if diff < margin and diff >= 0.25 and playerAstatpg > 4:
      df_ast_pairs = pd.concat([df_ast_pairs, pd.DataFrame.from_records([{
      "PlayerAId": pair[0].PlayerID,
      "PlayerAName": pair[0].Player,
      "PlayerAStatperGame": playerAstatpg,
      "PlayerATeam": pair[0].Team,
      "PlayerBId": pair[1].PlayerID,
      "PlayerBName": pair[1].Player,
      "PlayerBStatperGame": playerBstatpg,
      "PlayerBTeam": pair[1].Team,
      "SeasonStart": pair[1].SeasonStartYear,
      "Answer": answer,
      "StatCategory": "AST",
      "Interval": diff

      }])])

print(len(df_ast_pairs))
# df_ast_pairs.to_csv("AST_2022.csv")

3508


In [46]:
df_ast_pairs

,PlayerAId,PlayerAName,PlayerAStatperGame,PlayerATeam,PlayerBId,PlayerBName,PlayerBStatperGame,PlayerBTeam,SeasonStart,Answer,StatCategory,Interval
0,201935,James Harden,7.512821,HOU,202689,Kemba Walker,5.902439,CHA,2018,James Harden,AST,1.610381
0,201935,James Harden,7.512821,HOU,203078,Bradley Beal,5.463415,WAS,2018,James Harden,AST,2.049406
0,201935,James Harden,7.512821,HOU,203081,Damian Lillard,6.887500,POR,2018,James Harden,AST,0.625321
0,201935,James Harden,7.512821,HOU,201142,Kevin Durant,5.858974,GSW,2018,James Harden,AST,1.653846
0,201935,James Harden,7.512821,HOU,203507,Giannis Antetokounmpo,5.888889,MIL,2018,James Harden,AST,1.623932
...,...,...,...,...,...,...,...,...,...,...,...,...
0,204025,Tim Frazier,4.203390,MIL,101181,Jose Calderon,2.346939,DET,2018,Tim Frazier,AST,1.856451
0,204025,Tim Frazier,4.203390,MIL,1629121,Jaylen Adams,1.911765,ATL,2018,Tim Frazier,AST,2.291625
0,204025,Tim Frazier,4.203390,MIL,202738,Isaiah Thomas,1.916667,DEN,2018,Tim Frazier,AST,2.286723
0,204025,Tim Frazier,4.203390,MIL,2747,JR Smith,1.909091,CLE,2018,Tim Frazier,AST,2.294299


### Find pairs of players whose average REB/gm stats are similar based on a defined margin.

In [53]:
player_pairs = list(combinations(df_2223.itertuples(index=False), 2))
df_reb_pairs = pd.DataFrame()

margin = 1.4

for pair in player_pairs:
  playerAstatpg = pair[0].Rebounds / pair[0].GamesPlayed
  playerBstatpg = pair[1].Rebounds / pair[1].GamesPlayed
  if(pair[0].GamesPlayed > 10 and pair[1].GamesPlayed > 10):
    answer = pair[0].Player if playerAstatpg > playerBstatpg else pair[1].Player
    diff = abs(playerBstatpg - playerAstatpg)
    if diff < margin and diff > 0.2 and playerAstatpg > 5.2:
      df_reb_pairs = pd.concat([df_reb_pairs, pd.DataFrame.from_records([{
      "PlayerAId": pair[0].PlayerID,
      "PlayerAName": pair[0].Player,
      "PlayerAStatperGame": playerAstatpg,
      "PlayerATeam": pair[0].Team,
      "PlayerBId": pair[1].PlayerID,
      "PlayerBName": pair[1].Player,
      "PlayerBStatperGame": playerBstatpg,
      "PlayerBTeam": pair[1].Team,
      "SeasonStart": pair[1].SeasonStartYear,
      "Answer": answer,
      "StatCategory": "REB",
      "Interval": diff

      }])])

print(len(df_reb_pairs))
# df_reb_pairs.to_csv("AST_2022.csv)

2798


In [54]:
df_reb_pairs

,PlayerAId,PlayerAName,PlayerAStatperGame,PlayerATeam,PlayerBId,PlayerBName,PlayerBStatperGame,PlayerBTeam,SeasonStart,Answer,StatCategory,Interval
0,201935,James Harden,6.641026,HOU,201142,Kevin Durant,6.371795,GSW,2018,James Harden,REB,0.269231
0,201935,James Harden,6.641026,HOU,201939,Stephen Curry,5.347826,GSW,2018,James Harden,REB,1.293200
0,201935,James Harden,6.641026,HOU,201933,Blake Griffin,7.533333,DET,2018,Blake Griffin,REB,0.892308
0,201935,James Harden,6.641026,HOU,202699,Tobias Harris,7.865854,PHI,2018,Tobias Harris,REB,1.224828
0,201935,James Harden,6.641026,HOU,201942,DeMar DeRozan,6.000000,SAS,2018,James Harden,REB,0.641026
...,...,...,...,...,...,...,...,...,...,...,...,...
0,101162,Marcin Gortat,5.553191,LAC,101106,Andrew Bogut,5.000000,GSW,2018,Marcin Gortat,REB,0.553191
0,2199,Tyson Chandler,5.581818,LAL,201585,Kosta Koufos,4.214286,SAC,2018,Tyson Chandler,REB,1.367532
0,2199,Tyson Chandler,5.581818,LAL,2200,Pau Gasol,4.566667,MIL,2018,Tyson Chandler,REB,1.015152
0,2199,Tyson Chandler,5.581818,LAL,203089,John Henson,5.071429,CLE,2018,Tyson Chandler,REB,0.510390


###Firestore update for all stats

In [55]:
import os
from google.cloud import firestore
from google.colab import auth

auth.authenticate_user()

In [56]:
# Combine all resulting dfs
df_pra = pd.DataFrame()
df_pra = pd.concat([df_pts_pairs, df_ast_pairs, df_reb_pairs], ignore_index=True)

In [57]:
import random

queued_questions = 0
included_players = []
while queued_questions < 10:
  index = random.randint(0,len(df_pra))
  single_doc = df_pra.iloc[index].to_dict()
  if(single_doc["PlayerAName"] in included_players or single_doc["PlayerBName"] in included_players):
    pass
  else:
    included_players.append(single_doc["PlayerAName"])
    included_players.append(single_doc["PlayerBName"])
    project_id = "vue-nba-guess"
    db = firestore.Client(project=project_id)
    doc_ref = db.collection("daily-challenge").document("2023-07-27").collection("questions").document(str(queued_questions))
    doc_ref.set(single_doc)
    queued_questions = queued_questions + 1


###Firestore update for only AST

In [ ]:
import os
from google.cloud import firestore
from google.colab import auth

auth.authenticate_user()

In [ ]:
import random

queued_questions = 0
included_players = []
while queued_questions < 10:
  index = random.randint(0,len(df_ast_pairs))
  single_doc = df_ast_pairs.iloc[index].to_dict()
  if(single_doc["PlayerAName"] in included_players or single_doc["PlayerBName"] in included_players):
    pass
  else:
    included_players.append(single_doc["PlayerAName"])
    included_players.append(single_doc["PlayerBName"])
    project_id = "vue-nba-guess"
    db = firestore.Client(project=project_id)
    doc_ref = db.collection("daily-challenge").document("2023-07-27").collection("questions").document(str(queued_questions))
    # doc_ref.set(single_doc)
    queued_questions = queued_questions + 1


<br>